In [5]:
import json
import requests

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

from langchain.agents import create_agent


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

prompt = """
Analiza el siguiente problema y responde con:
- un resumen
- los riesgos principales
- un nivel de confianza

Problema:
La empresa depende de un único proveedor de infraestructura en la nube.
"""

response = llm.invoke(prompt)
print(response.content)

**Resumen:**
La empresa está en una situación de dependencia crítica al contar con un solo proveedor de infraestructura en la nube. Esto significa que todos sus servicios, aplicaciones y datos están alojados y gestionados por un único proveedor, lo que puede tener implicaciones significativas en términos de operatividad, costos y seguridad.

**Riesgos principales:**

1. **Riesgo de interrupción del servicio:** Si el proveedor experimenta una interrupción o falla técnica, la empresa podría enfrentar tiempos de inactividad significativos, afectando su capacidad para operar y prestar servicios a sus clientes.

2. **Riesgo financiero:** El proveedor podría aumentar sus precios, lo que impactaría directamente en los costos operativos de la empresa. Sin alternativas, la empresa tendría que aceptar los nuevos términos o enfrentar costos y esfuerzos significativos para migrar a otro proveedor.

3. **Riesgo de seguridad y cumplimiento:** La dependencia de un solo proveedor puede limitar la capa

In [7]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
# -------------------------
# Modelo Pydantic (contrato)
# -------------------------
class ConfidenceLevel(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"


class AnalysisResult(BaseModel):
    summary: str = Field(description="Resumen claro del problema")
    risks: List[str] = Field(description="Lista de riesgos identificados")
    confidence: ConfidenceLevel


# -------------------------
# Parser
# -------------------------
parser = PydanticOutputParser(pydantic_object=AnalysisResult)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un analista de riesgos.\n"
            "Debes responder EXCLUSIVAMENTE en el formato indicado.\n"
            "{format_instructions}",
        ),
        ("human", "{problem}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

print('Instrucciones del parser: ', parser.get_format_instructions())

llm = ChatOpenAI(model="gpt-4o", temperature=0)

chain = prompt | llm | parser


# -------------------------
# Ejecución
# -------------------------
result = chain.invoke(
    {
        "problem": "La empresa depende de un único proveedor de infraestructura en la nube."
    }
)

print(result,'\n')
print('Summary: ',result.summary)
print('Risks: ')
for risk in result.risks:
    print('-', risk)
print('Confidence: ', result.confidence)


Instrucciones del parser:  The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"ConfidenceLevel": {"enum": ["low", "medium", "high"], "title": "ConfidenceLevel", "type": "string"}}, "properties": {"summary": {"description": "Resumen claro del problema", "title": "Summary", "type": "string"}, "risks": {"description": "Lista de riesgos identificados", "items": {"type": "string"}, "title": "Risks", "type": "array"}, "confidence": {"$ref": "#/$defs/ConfidenceLevel"}}, "required": ["summary", "risks", "confidence"]}
```
summary='La empresa depende de un único proveedor de infraestructura 